In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics  import f1_score
from sklearn.pipeline import make_pipeline
from sklearn import metrics

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/Colab Notebooks/230219-DataHackathon


Mounted at /content/gdrive
[Errno 2] No such file or directory: '/content/gdrive/MyDrive/Colab Notebooks/230219-DataHackathon'
/content


# New Section

In [ ]:
cyclica = pd.read_csv("sample5000.csv")

FileNotFoundError: ignored

In [ ]:
cyclica.head()
cyclica.columns

Index(['Unnamed: 0', 'annotation_sequence', 'feat_A', 'feat_C', 'feat_D',
       'feat_E', 'feat_F', 'feat_G', 'feat_H', 'feat_I', 'feat_K', 'feat_L',
       'feat_M', 'feat_N', 'feat_P', 'feat_Q', 'feat_R', 'feat_S', 'feat_T',
       'feat_V', 'feat_W', 'feat_Y', 'annotation_atomrec', 'feat_PHI',
       'feat_PSI', 'feat_TAU', 'feat_THETA', 'feat_BBSASA', 'feat_SCSASA',
       'feat_pLDDT', 'feat_DSSP_H', 'feat_DSSP_B', 'feat_DSSP_E',
       'feat_DSSP_G', 'feat_DSSP_I', 'feat_DSSP_T', 'feat_DSSP_S',
       'feat_DSSP_6', 'feat_DSSP_7', 'feat_DSSP_8', 'feat_DSSP_9',
       'feat_DSSP_10', 'feat_DSSP_11', 'feat_DSSP_12', 'feat_DSSP_13',
       'coord_X', 'coord_Y', 'coord_Z', 'entry', 'entry_index', 'y_Ligand'],
      dtype='object')

In [ ]:
cyclica_drop = cyclica.drop(labels=["Unnamed: 0", "entry_index"], axis=1, inplace=False)
cyclica_drop.head()

,annotation_sequence,feat_A,feat_C,feat_D,feat_E,feat_F,feat_G,feat_H,feat_I,feat_K,...,feat_DSSP_9,feat_DSSP_10,feat_DSSP_11,feat_DSSP_12,feat_DSSP_13,coord_X,coord_Y,coord_Z,entry,y_Ligand
0,G,False,False,False,False,False,True,False,False,False,...,-2.4,1,-0.1,0,0.0,42.931999,13.354000,-44.346001,FGFR2_HUMAN,False
1,Q,False,False,False,False,False,False,False,False,False,...,-0.3,-6,-0.2,-1,-0.2,37.630001,5.143000,-27.915001,EGFR_HUMAN,False
2,A,True,False,False,False,False,False,False,False,False,...,-2.0,1,-0.2,-1,-0.2,-4.711000,-30.237000,16.455000,KIF5A_HUMAN,False
3,S,False,False,False,False,False,False,False,False,False,...,0.0,2,-0.0,0,0.0,-42.396000,-65.049004,90.838997,CASR_HUMAN,False
4,L,False,False,False,False,False,False,False,False,False,...,-2.9,-2,-0.4,2,-0.8,1.453000,11.417000,9.808000,KPYM_HUMAN,False


In [ ]:
# cyclica_drop.entry.str.extract(pat = r'\w+', expand=False)

In [ ]:
cyclica.annotation_atomrec.isna().sum()

132

In [ ]:
# cyclica.annotation_atomrec.fillna(method="mode")
mode = cyclica_drop.annotation_atomrec.mode()[0]
cyclica_drop["annotation_atomrec"] = cyclica_drop.annotation_atomrec.fillna(value=mode)

In [ ]:
cyclica_drop.annotation_atomrec.isna().sum()

0

Data preprocessing finishes

# Model Prototyping

In [ ]:
cyclica_no_entry = cyclica_drop.drop(labels="entry", axis=1)

In [ ]:
unique_annotation = cyclica_no_entry.annotation_atomrec.unique()
unique_annotation.sort()
unique_annotation

array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P',
       'Q', 'R', 'S', 'T', 'V', 'W', 'Y'], dtype=object)

In [ ]:
annotation_encoding = np.arange(20)
amino_encoded = {}
for key, val in zip(unique_annotation, annotation_encoding):
    amino_encoded[key] = val

amino_encoded

{'A': 0,
 'C': 1,
 'D': 2,
 'E': 3,
 'F': 4,
 'G': 5,
 'H': 6,
 'I': 7,
 'K': 8,
 'L': 9,
 'M': 10,
 'N': 11,
 'P': 12,
 'Q': 13,
 'R': 14,
 'S': 15,
 'T': 16,
 'V': 17,
 'W': 18,
 'Y': 19}

In [ ]:
cyclica_no_entry.annotation_atomrec = cyclica_no_entry.annotation_atomrec.map(amino_encoded)
cyclica_no_entry.annotation_sequence = cyclica_no_entry.annotation_sequence.map(amino_encoded)

In [ ]:
X, y = cyclica_no_entry.drop(labels="y_Ligand", axis=1), cyclica_no_entry["y_Ligand"]
X_train, X_test, y_train, y_test = \
    train_test_split(X, y,
                    train_size=4000,
                    test_size=1000,
                    random_state=226)

In [ ]:
X.columns

Index(['annotation_sequence', 'feat_A', 'feat_C', 'feat_D', 'feat_E', 'feat_F',
       'feat_G', 'feat_H', 'feat_I', 'feat_K', 'feat_L', 'feat_M', 'feat_N',
       'feat_P', 'feat_Q', 'feat_R', 'feat_S', 'feat_T', 'feat_V', 'feat_W',
       'feat_Y', 'annotation_atomrec', 'feat_PHI', 'feat_PSI', 'feat_TAU',
       'feat_THETA', 'feat_BBSASA', 'feat_SCSASA', 'feat_pLDDT', 'feat_DSSP_H',
       'feat_DSSP_B', 'feat_DSSP_E', 'feat_DSSP_G', 'feat_DSSP_I',
       'feat_DSSP_T', 'feat_DSSP_S', 'feat_DSSP_6', 'feat_DSSP_7',
       'feat_DSSP_8', 'feat_DSSP_9', 'feat_DSSP_10', 'feat_DSSP_11',
       'feat_DSSP_12', 'feat_DSSP_13', 'coord_X', 'coord_Y', 'coord_Z'],
      dtype='object')

In [ ]:
## Logistic classifier
logit_clf = LogisticRegression(random_state=226, penalty='l2', tol=0.001, solver = 'liblinear')
logit_clf = logit_clf.fit(X_train, y_train)

y_prob = logit_clf.predict_proba(X_test)
y_pred = logit_clf.predict(X_test)
f1_loss = f1_score(y_test, y_pred, average='micro')
accuracy = metrics.accuracy_score(y_test, y_pred) 
conf_mtrx = metrics.confusion_matrix(y_test.tolist(), y_pred.tolist(), normalize= 'true')
true_pos_rate = conf_mtrx[1,1]
true_neg_rate = conf_mtrx[0,0]


fpr, tpr, thresholds = metrics.roc_curve(y_test, y_prob[:, 1], pos_label = 1)
AUC_score = metrics.auc(fpr, tpr)

# performance_df = pd.DataFrame({
#     "F1_Loss":f1_loss,
#     "Accuracy": accuracy,
#     "True Pos Rate": true_pos_rate,
#     "True Negative Rate": true_neg_rate,
#     "AUC": AUC_score
# })


In [ ]:
performance_df = pd.DataFrame({
    "F1_Loss":f1_loss,
    "Accuracy": accuracy,
    "True Pos Rate": true_pos_rate,
    "True Negative Rate": true_neg_rate,
    "AUC": AUC_score
}, index = ["Logistic Full"])
performance_df

,F1_Loss,Accuracy,True Pos Rate,True Negative Rate,AUC
Logistic Full,0.965,0.965,0.0,1.0,0.687787


In [ ]:
## randomForest using gini index
RFclf_gini = RandomForestClassifier(max_features="sqrt",
                                    bootstrap=False,
                                min_samples_leaf= 5,
                                random_state=226)
RFclf_gini = RFclf_gini.fit(X_train, y_train)

y_prob = RFclf_gini.predict_proba(X_test)
y_pred = RFclf_gini.predict(X_test)

f1_loss = f1_score(y_test, y_pred, average='micro')
accuracy = metrics.accuracy_score(y_test, y_pred) 

conf_mtrx = metrics.confusion_matrix(y_test.tolist(), y_pred.tolist(), normalize= 'true')
true_pos_rate = conf_mtrx[1,1]
true_neg_rate = conf_mtrx[0,0]

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_prob[:, 1], pos_label = 1)
AUC_score = metrics.auc(fpr, tpr)


In [ ]:
performance_df.loc["RandomForest_Gini"] = [f1_loss, accuracy, true_pos_rate, true_neg_rate, AUC_score]
performance_df

,F1_Loss,Accuracy,True Pos Rate,True Negative Rate,AUC
Logistic Full,0.965,0.965,0.0,1.0,0.687787
RandomForest_Gini,0.965,0.965,0.0,1.0,0.809874


In [ ]:
## randomForest using entropy
RFclf_entropy = RandomForestClassifier(max_features= 'sqrt',
                                       criterion="entropy",
                                    bootstrap=False,
                                min_samples_leaf= 5,
                                random_state=226)
RFclf_entropy = RFclf_entropy.fit(X_train, y_train)

y_prob = RFclf_entropy.predict_proba(X_test)
y_pred = RFclf_entropy.predict(X_test)

f1_loss = f1_score(y_test, y_pred, average='micro')
accuracy = metrics.accuracy_score(y_test, y_pred) 

conf_mtrx = metrics.confusion_matrix(y_test.tolist(), y_pred.tolist(), normalize= 'true')
true_pos_rate = conf_mtrx[1,1]
true_neg_rate = conf_mtrx[0,0]

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_prob[:, 1], pos_label = 1)
AUC_score = metrics.auc(fpr, tpr)

performance_df.loc["RandomForest_Entropy"] = [f1_loss, accuracy, true_pos_rate, true_neg_rate, AUC_score]
performance_df
## Quite similar result but better AUC than logistic regression

,F1_Loss,Accuracy,True Pos Rate,True Negative Rate,AUC
Logistic Full,0.965,0.965,0.0,1.0,0.687787
RandomForest_Gini,0.965,0.965,0.0,1.0,0.809874
RandomForest_Entropy,0.965,0.965,0.0,1.0,0.808320


In [ ]:
## randomForest using log loss
RFclf_logloss = RandomForestClassifier(max_features= 'sqrt',
                                       criterion="log_loss",
                                    bootstrap=False,
                                min_samples_leaf= 5,
                                random_state=226)
RFclf_logloss = RFclf_logloss.fit(X_train, y_train)

y_prob = RFclf_logloss.predict_proba(X_test)
y_pred = RFclf_logloss.predict(X_test)

f1_loss = f1_score(y_test, y_pred, average='micro')
accuracy = metrics.accuracy_score(y_test, y_pred) 

conf_mtrx = metrics.confusion_matrix(y_test.tolist(), y_pred.tolist(), normalize= 'true')
true_pos_rate = conf_mtrx[1,1]
true_neg_rate = conf_mtrx[0,0]

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_prob[:, 1], pos_label = 1)
AUC_score = metrics.auc(fpr, tpr)

performance_df.loc["RandomForest_logloss"] = [f1_loss, accuracy, true_pos_rate, true_neg_rate, AUC_score]
performance_df
## Quite similar result but better AUC than logistic regression

KeyError: ignored

In [ ]:
# AdaBoost
ada_clf = AdaBoostClassifier(n_estimators=100, random_state=226)
ada_clf = ada_clf.fit(X_train, y_train)

y_prob = ada_clf.predict_proba(X_test)
y_pred = ada_clf.predict(X_test)

f1_loss = f1_score(y_test, y_pred, average='micro')
accuracy = metrics.accuracy_score(y_test, y_pred) 

conf_mtrx = metrics.confusion_matrix(y_test.tolist(), y_pred.tolist(), normalize= 'true')
true_pos_rate = conf_mtrx[1,1]
true_neg_rate = conf_mtrx[0,0]

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_prob[:, 1], pos_label = 1)
AUC_score = metrics.auc(fpr, tpr)

performance_df.loc["AdaBoost"] = [f1_loss, accuracy, true_pos_rate, true_neg_rate, AUC_score]
performance_df
## Quite similar result but better AUC than logistic regression

,F1_Loss,Accuracy,True Pos Rate,True Negative Rate,AUC
Logistic Full,0.965,0.965,0.000000,1.000000,0.687787
RandomForest_Gini,0.965,0.965,0.000000,1.000000,0.809874
RandomForest_Entropy,0.965,0.965,0.000000,1.000000,0.808320
AdaBoost,0.962,0.962,0.028571,0.995855,0.747565


In [ ]:
performance_df.to_csv("230219-junior-test-data.csv")

In [ ]:
cyclica_no_entry.to_csv("230219-cyclicaNoEntry.csv")